### Step 1 - Data Cleaning

Open all the recipes data in one file and prepare them for further analysis!

In [2]:
# Import the necessary library!

import pandas as pd
import os
import numpy as np
import glob
import datetime

import nltk
import re
import string
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
print('package ready')

package ready


In [3]:
# List all the files in the folder!  
files = glob.glob('guardian/*.csv')
# read the files in a dataframe  
data_df = [pd.read_csv(f) for f in files]

In [4]:
# Append all the files together  
data = pd.concat(data_df,ignore_index=True)
# Reset the index  
data = data.reset_index()  
# Drop the duplicates  
data = data.drop_duplicates()

In [7]:
print("Number of data (recipes) : " + str(data.shape))
print()
data.head()

Number of data (recipes) : (7603, 11)



,index,Unnamed: 0,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url
0,0,0.0,2009-04-12T00:01:00+0100,My space,My space,Anna Chapman,The passionate entrepreneur Willy Harcourt-Coo...,"E\night years ago, my wife Tania and I came ba...",No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
1,1,1.0,2009-04-12T00:01:00+0100,Nigel Slater's Easter eggs,Nigel Slater recipes,Nigel Slater,Nothing comes with a trickier reputation to ma...,I\nt is fair to say the eggs I am most interes...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...
2,2,2.0,2009-04-11T00:01:00+0100,Indulge the family for under a tenner,Family,Annalisa Barbieri,If you fancy treating yourself and your family...,Get building\nWhen was the last time you made ...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...
3,3,3.0,2009-04-11T00:01:00+0100,How to bake,How to bake,Dan Lepard,Lamb and parsley pie,A good dish for Easter Sunday. The star is the...,No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
4,4,4.0,2009-04-11T00:01:00+0100,Rising stars,Hugh Fearnley-Whittingstall recipes,Hugh Fearnley-Whittingstall,Easter is traditionally a time for showing off...,I\n'm celebrating Easter this week with a bit ...,No Comments,1,https://www.theguardian.com/lifeandstyle/2009/...


In [9]:
# Drop the column that we don't need
data = data.drop(columns=['Unnamed: 0','index'])

KeyError: "['Unnamed: 0' 'index'] not found in axis"

In [10]:
# Look for null value in the data
print(data.isnull().sum())

date                 0
headline_list       28
category            35
author             349
subtitle           210
article_content      1
comments_number      3
share_number         3
url                  2
dtype: int64


In [11]:
# Make a bit of cleaning

# Delete all the article with category 'Family Life' - Not relevant for recipes analysis! 
data = data[data['category']!='Family life']

In [ ]:
# Check at the name of the author
for i in data.author:
    print(i)

In [12]:
# Replace the author that have been wrongly webscraped!  
data = data.replace({'[<meta content="Dale Berning Sawa" name="author"/>]':'Dale Berning Sawa',  
                     '[<meta content="Chi-chi Nwanoku" name="author"/>]':'Chi-chi Nwanoku',  
                     '[<meta content="Hugh Fearnley-Whittingstall" name="author"/>]':'Hugh Fearnley-Whittingstall',  
                     '[<meta content="Jeanette Winterson" name="author"/>]':'Jeanette Winterson',  
                     '[<meta content="Matthew Fort" name="author"/>]':'Matthew Fort',  
                     '[<meta content="Susan McCarthy" name="author"/>]':'Susan McCarthy',  
                     '[<meta content="Ariane Sherine" name="author"/>]':'Ariane Sherine',  
                     '[<meta content="Ruby Tandoh" name="author"/>]':'Ruby Tandoh',  
                     '[<meta content="the Guardian" name="author"/>]':'the Guardian',  
                     '[<meta content="Romy Ash" name="author"/>, <meta content="Sarah Trotter" name="author"/>, <meta content="Romy Ash" name="author"/>, <meta content="Lauren Bamford" name="author"/>, <meta content="Sarah Trotter" name="author"/>]':'Romy Ash, Sara Trotter, Lauren Bamford',  
                     '[<meta content="Clem Bastow" name="author"/>]':'Clem Bastow',  
                     '[<meta content="Yotam Ottolenghi" name="author"/>, <meta content="Uyen Luu" name="author"/>, <meta content="David Frenkiel" name="author"/>, <meta content="Luise Vindahl" name="author"/>, <meta content="Caroline Craig" name="author"/>]':'Caroline Craig, Luise Vindahl, Yotam Ottolenghi, Uyen Luu, David Frenkiel',  
                     'Yotam Ottlenghi':'Yotam Ottolenghi','Yuki Sugiura, Valerie Berry, Lee Gould, Rachel Vere and Music by Evan Gildersleeve, theguardian.com':'Yuki Sugiura, Valerie Berry, Lee Gould, Rachel Vere'})

# If NaN for authors and NaN for subtitle -> change it to none!   
values = {'author': 'none', 'subtitle':'none'}  
data = data.fillna(value=values)

In [13]:
# Check for the no comments and no url!
test = data[data['comments_number'].isnull()]  
test.share_number = 'No Sharing'  
data.append(test)

change = data[(data['date'] == '2015-01-20') & (data['author']== 'Ruby Tandoh' )]  
change.share_number = 52  
change.url = 'https://www.theguardian.com/lifeandstyle/2015/jan/20/best-baking-recipes-from-2014-ruby-tandoh'  
change.comments_number = 25  
data.append(change)

/Users/audevuilliomenet/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url
0,2009-04-12T00:01:00+0100,My space,My space,Anna Chapman,The passionate entrepreneur Willy Harcourt-Coo...,"E\night years ago, my wife Tania and I came ba...",No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
1,2009-04-12T00:01:00+0100,Nigel Slater's Easter eggs,Nigel Slater recipes,Nigel Slater,Nothing comes with a trickier reputation to ma...,I\nt is fair to say the eggs I am most interes...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...
2,2009-04-11T00:01:00+0100,Indulge the family for under a tenner,Family,Annalisa Barbieri,If you fancy treating yourself and your family...,Get building\nWhen was the last time you made ...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...
3,2009-04-11T00:01:00+0100,How to bake,How to bake,Dan Lepard,Lamb and parsley pie,A good dish for Easter Sunday. The star is the...,No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
4,2009-04-11T00:01:00+0100,Rising stars,Hugh Fearnley-Whittingstall recipes,Hugh Fearnley-Whittingstall,Easter is traditionally a time for showing off...,I\n'm celebrating Easter this week with a bit ...,No Comments,1,https://www.theguardian.com/lifeandstyle/2009/...
5,2009-04-11T00:01:00+0100,Indulge the family for under a tenner,Family,Annalisa Barbieri,If you fancy treating yourself and your family...,Get building\nWhen was the last time you made ...,No Comments,1,https://www.theguardian.com/lifeandstyle/2009/...
6,2009-04-08T10:16:18+0100,Get your buns out,Word of Mouth blog,Dan Lepard,Let's bake ourselves into a frenzy this Easter...,I\nwas asked by a lovely magazine called Resur...,19,0,https://www.theguardian.com/lifeandstyle/wordo...
7,2009-04-08T00:01:00+0100,The G2 weekly recipe: Trout and spinach frittata,The G2 weekly recipe,Allegra McEvedy,none,N\now that we're allowed to eat as many eggs a...,No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
8,2009-04-07T12:00:00+0100,Skip the lamb this Easter,Word of Mouth blog,Tim Hayward,Forget the lamb for your celebratory meal and ...,"A\nah Easter. the sap is rising, plants are do...",37,1,https://www.theguardian.com/lifeandstyle/wordo...
9,2009-04-06T00:01:00+0100,This year's tastiest Easter eggs,Taste test,none,none,This video is no longer available.\nLast week ...,No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...


In [18]:
# Remove the rows if NaN is in url, headline_list, share_number and category after having make the above change!  
data = data.dropna(subset = ['headline_list','url','share_number','category'])  

print('Number of rows (recipes), data clean: ' + str(data.shape))
print("")
print('Data Clean')

In [27]:
# Check the types of the data!
data.dtypes

date               object
headline_list      object
category           object
author             object
subtitle           object
article_content    object
comments_number    object
share_number       object
url                object
dtype: object

In [32]:
data.head(2)

,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url
0,2009-04-12,My space,My space,Anna Chapman,The passionate entrepreneur Willy Harcourt-Coo...,"E\night years ago, my wife Tania and I came ba...",No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...
1,2009-04-12,Nigel Slater's Easter eggs,Nigel Slater recipes,Nigel Slater,Nothing comes with a trickier reputation to ma...,I\nt is fair to say the eggs I am most interes...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...


Change the data date to date instead of object!

In [31]:
#Separe the date in year and month
# Remove the strings of time in date. Keep only the day and the month.b  
data.date = data.date.str.replace('T\d.*','')  
data['date'] = pd.to_datetime(data['date'])

# Add a column in dataframe for the year, month and day  
data['year'] = data['date'].dt.year  
data['month'] = data['date'].dt.month_name()  
data['day'] = data['date'].dt.day 

In [34]:
data.head(2)

,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url,year,month,day
0,2009-04-12,My space,My space,Anna Chapman,The passionate entrepreneur Willy Harcourt-Coo...,"E\night years ago, my wife Tania and I came ba...",No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...,2009,April,12
1,2009-04-12,Nigel Slater's Easter eggs,Nigel Slater recipes,Nigel Slater,Nothing comes with a trickier reputation to ma...,I\nt is fair to say the eggs I am most interes...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...,2009,April,12


In [35]:
data.dtypes

date               datetime64[ns]
headline_list              object
category                   object
author                     object
subtitle                   object
article_content            object
comments_number            object
share_number               object
url                        object
year                        int64
month                      object
day                         int64
dtype: object

In [131]:
## Function for tokenization and cleaning text!   
cleaning = ['headline_list','subtitle','article_content']  
stop = stopwords.words('english')  
guardian1 = 'since you‚Äôre here‚Ä¶ ‚Ä¶ we have a small favour to ask more people around the world are reading the guardian‚Äôs independent investigative journalism than ever before we‚Äôve now been funded by over one million readers and unlike many news organisations we have chosen an approach that allows us to keep our journalism open to all we believe that each one of us deserves access to accurate information with integrity at its heart the guardian is editorially independent meaning we set our own agenda our journalism is free from commercial bias and not influenced by billionaire owners politicians or shareholders no one edits our editor no one steers our opinion this is important as it enables us to give a voice to those less heard challenge the powerful and hold them to account it‚Äôs what makes us different to so many others in the media at a time when factual honest reporting is critical every contribution we receive from readers like you big or small goes directly into funding our journalism this support enables us to keep working as we do ‚Äì but we must maintain and build on it for every year to come support the guardian from as little as ¬£1 ‚Äì and it only takes a minute thank you support the guardian' 

In [135]:
def cleandata(data):
        for i in cleaning:
                # Remove the punctuation  
                data['clean_'+str(i)] = data[i].apply(lambda x:x.translate(str.maketrans('','', string.punctuation)))  
                print('step 1: punctuation remove: ' + str(i))

                # Remove the '\\n' line_brakes  
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'\n',' ')
                data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(r'Since you’re here.*','')
                if i == 'article_content':
                        data['clean_'+str(i)] = data['clean_'+str(i)].str.replace(guardian1,'')
                else:
                        continue
                # Get all the words in lowercase  
                data['clean_'+str(i)] = data['clean_'+str(i)].str.lower()  
                print('step 2: words in lowercase')

                # Remove all the english stopwords!  
                data['nostop_words_'+'clean_'+str(i)] = data['clean_'+str(i)].apply(  
                lambda x: ' '.join([word for word in x.split() if word not in (stop)]))  
                # Remove the special characters!  
                data['nostop_words_'+'clean_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].str.replace(r"\’|\s\–|\'",'')                                                                                    
                data['nostop_words_'+'clean_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].str.replace(r'[0-9]+', '')
                
                print('step 3: nonstopwords and special characters removed')

                # Tokenize the text  
                data['token_'+str(i)] = data['nostop_words_'+'clean_'+str(i)].apply(word_tokenize)
                print('step 4: words tokenize')
                print('Data Clean Finish')
        return

cleandata(data)

punctuation remove headline_list
punctuation remove subtitle
punctuation remove article_content
words in lowercase
nonstopwords and special characters removed
words tokenize
Data Clean Finish


In [140]:
data.head(2)

,date,headline_list,category,author,subtitle,article_content,comments_number,share_number,url,year,month,day,clean_headline_list,clean_subtitle,clean_article_content,nostop_words_clean_article_content,token_article_content
0,2009-04-12,My space,My space,Anna Chapman,The passionate entrepreneur Willy Harcourt-Coo...,"E\night years ago, my wife Tania and I came ba...",No Comments,0,https://www.theguardian.com/lifeandstyle/2009/...,2009,April,12,My space,The passionate entrepreneur Willy HarcourtCooz...,e ight years ago my wife tania and i came back...,e ight years ago wife tania came back venezuel...,"[e, ight, years, ago, wife, tania, came, back,..."
1,2009-04-12,Nigel Slater's Easter eggs,Nigel Slater recipes,Nigel Slater,Nothing comes with a trickier reputation to ma...,I\nt is fair to say the eggs I am most interes...,No Comments,No Sharing,https://www.theguardian.com/lifeandstyle/2009/...,2009,April,12,Nigel Slaters Easter eggs,Nothing comes with a trickier reputation to ma...,i t is fair to say the eggs i am most interest...,fair say eggs interested come weekend wrapped ...,"[fair, say, eggs, interested, come, weekend, w..."


In [144]:
# Save the Clean Text in a CSV file
data.to_csv('guardian_clean_data.csv')
print('data clean and save in csv file')

data clean and save in csv file
